In [1]:
import os
import sys
from tensorflow.keras import models
import tensorflow as tf
import numpy as np
import json

parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
sys.path.append(parent_dir)

# Import trained model

In [2]:
model_path = os.path.join(parent_dir, 'train', 'models')
model = models.load_model(os.path.join(model_path, 'Dense_seqpad.h5'))

In [3]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [4]:
config_file = os.path.join(model_path, 'configure.json')
with open(config_file, 'r', encoding='utf8') as fp:
    config = json.load(fp)
    
    
labels = dict()
for k,v in config.get('labels').items():
    v = int(v)
    if v in labels:
        labels[v] = f'{labels[v]}/{k}' # OK/OTP
    else:
        labels[v] = k

In [5]:
labels

{0: 'OK/OTP', 2: 'scam', 1: 'spam'}

In [6]:
examples = [
    'รหัส OTP สำหรับการยืนยันตัวตนของคุณสำหรับการใช้งาน GHB All คือ 318266 หมายเลขอ้างอิง IDN239S',
    'คุณได้SMSนี้ รับ100฿ ไม่ต้องแชร์ คลิก https://llonp9.co/n/8cCwo5mB',
    'รหัส OTP เพื่อยืนยันการปรับวงเงินการทำรายการรายวันของคุณคือ 97032 ห้ามบอกรหัสนี้แก่บุคคลอื่น',
    'มีการเปิดใช้งาน (Activate) MyMo เมื่อวันที่ 20/08/64 เวลา 14.32 น. หากท่านไม่ได้ทำรายการโปรดติดต่อ 1143 ทันที',
    'เนื่องจากคุณเป็นผู้ใช้ติ๊กต็อกที่มีประวัติที่ดี คุณได้รับเงินสดจากทาง ติ๊กต็อก แอดLine: wtt9',
    'รายการสั่งซื้อของท่าน IKEA หมายเลข W017-21112221 ของท่านจะถูกจัดส่งโดย MW_ST9997 STORE ในวันที่ 19-01-2023  ในช่วงเวลา  09:00 -12:00.หากมีข้อสงสัยกรุณาสอบถาม 02 708 7999 หรือ อีเมล์ customerrelations.ikeath@ikano.asia.Your IKEA order W017-21112221 will be delivered by MW_ST9997 STORE on 19-01-2023  between  09:00 -12:00.Need help Call 02 708 7999 or email customerrelations.ikeath@ikano.asia',
    'KINGKONG555 ไม่แตกเว็บอื่น มาลองเว็บเรา กำลังแจก https://bit.ly/kk557',
    'ปรับค่าแตกให้สูงกว่า เฉพาะสิ้นเดือนเท่านั้น คลิก bit.ly/3LeuScU',
    'PGGOLD แตกหนักไม่เกรงใจใคร ฝากมาเลย แตกจริง99% https://bit.ly/glo33',
    'Weplay168 ไม่แตกเว็บอื่นมาลองเว็บเรากำลังแจก ด่วน https://bit.ly/wev33',
    'ยูสใหม่โอกาสชนะกว่า90% เว็Uตรงไม่อั้นถอน https://bit.ly/pao13',
    'ถอนกี่ครั้งก็ได้ ถอนได้ไม่อั้น เว็บตรง Pao168 https://bit.ly/pao13',
]

# Predict

In [7]:
from helper import textutils

3.0.8


## Example of Sequence and Padding

In [8]:
for txt in examples:
    sanitized_txt = textutils.sanitize_text(txt)
    textrep_array = textutils.padded_seq_from_text(sanitized_txt, 
                                                config['word_index'], 
                                                config['max_length'], 
                                                ignore_err=True) 
    
    print (txt)
    print (sanitized_txt)
    print (textutils.tokenize(sanitized_txt, clean=True))
    print ()
    
    predictions = probability_model.predict([textrep_array])
    predict = np.argmax(predictions[0])
    
    print (labels[predict])
    print ()    

Found word not in word_index: word=idn


รหัส OTP สำหรับการยืนยันตัวตนของคุณสำหรับการใช้งาน GHB All คือ 318266 หมายเลขอ้างอิง IDN239S
รหัส OTP สำหรับการยืนยันตัวตนของคุณสำหรับการใช้งาน GHB All คือ 318266 หมายเลขอ้างอิง IDN239S
['รหัส', 'otp', 'สำหรับ', 'การ', 'ยืนยัน', 'ตัวตน', 'ของ', 'คุณ', 'สำหรับ', 'การใช้งาน', 'ghb', 'all', 'คือ', 'หมายเลข', 'อ้างอิง', 'idn', 's']

1/1 [==============================] - 0s 132ms/step
OK/OTP

คุณได้SMSนี้ รับ100฿ ไม่ต้องแชร์ คลิก https://llonp9.co/n/8cCwo5mB
คุณได้SMSนี้ รับ100฿ ไม่ต้องแชร์ คลิก <LINK>
['คุณ', 'ได้', 'sms', 'นี้', 'รับ', 'บาท', 'ไม่ต้อง', 'แชร์', 'คลิก', '<LINK>']

1/1 [==============================] - 0s 38ms/step
scam

รหัส OTP เพื่อยืนยันการปรับวงเงินการทำรายการรายวันของคุณคือ 97032 ห้ามบอกรหัสนี้แก่บุคคลอื่น
รหัส OTP เพื่อยืนยันการปรับวงเงินการทำรายการรายวันของคุณคือ 97032 ห้ามบอกรหัสนี้แก่บุคคลอื่น
['รหัส', 'otp', 'เพื่อ', 'ยืนยัน', 'การ', 'ปรับ', 'วงเงิน', 'การ', 'ทำ', 'รายการ', 'รายวัน', 'ของ', 'คุณ', 'คือ', 'ห้าม', 'บอก', 'รหัส', 'นี้', 'แก่', 'บุคคล', 'อื่น']

1/

Found word not in word_index: word=wtt


OK/OTP

เนื่องจากคุณเป็นผู้ใช้ติ๊กต็อกที่มีประวัติที่ดี คุณได้รับเงินสดจากทาง ติ๊กต็อก แอดLine: wtt9
เนื่องจากคุณเป็นผู้ใช้ติ๊กต็อกที่มีประวัติที่ดี คุณได้รับเงินสดจากทาง ติ๊กต็อก แอดLine: wtt9
['เนื่องจาก', 'คุณ', 'เป็น', 'ผู้ใช้', 'ติ๊ก', 'ต็อก', 'ที่', 'มีประวัติ', 'ที่', 'ดี', 'คุณ', 'ได้รับ', 'เงินสด', 'จาก', 'ทาง', 'ติ๊ก', 'ต็อก', 'แอด', 'line', 'wtt']

1/1 [==============================] - 0s 41ms/step


Found word not in word_index: word=mw
Found word not in word_index: word=st
Found word not in word_index: word=delivered
Found word not in word_index: word=mw
Found word not in word_index: word=st
Found word not in word_index: word=between


...

scam

PGGOLD แตกหนักไม่เกรงใจใคร ฝากมาเลย แตกจริง99% https://bit.ly/glo33
PGGOLD แตกหนักไม่เกรงใจใคร ฝากมาเลย แตกจริง99% <SHORT-LINK>
['pggold', 'แตก', 'หนัก', 'ไม่', 'เกรงใจ', 'ใคร', 'ฝาก', 'มา', 'เลย', 'แตก', 'จริง', '<SHORT-LINK>']

1/1 [==============================] - 0s 37ms/step


Found word not in word_index: word=weplay


scam

Weplay168 ไม่แตกเว็บอื่นมาลองเว็บเรากำลังแจก ด่วน https://bit.ly/wev33
Weplay168 ไม่แตกเว็บอื่นมาลองเว็บเรากำลังแจก ด่วน <SHORT-LINK>
['weplay', 'ไม่', 'แตก', 'เว็บ', 'อื่น', 'มา', 'ลอง', 'เว็บ', 'เรา', 'กำลัง', 'แจก', 'ด่วน', '<SHORT-LINK>']

1/1 [==============================] - 0s 38ms/step
scam

ยูสใหม่โอกาสชนะกว่า90% เว็Uตรงไม่อั้นถอน https://bit.ly/pao13
ยูสใหม่โอกาสชนะกว่า90% เว็Uตรงไม่อั้นถอน <SHORT-LINK>
['ยูส', 'ใหม่', 'โอกาส', 'ชนะ', 'กว่า', 'เว็', 'u', 'ตรง', 'ไม่', 'อั้น', 'ถอน', '<SHORT-LINK>']

1/1 [==============================] - 0s 40ms/step


Found word not in word_index: word=pao


scam

ถอนกี่ครั้งก็ได้ ถอนได้ไม่อั้น เว็บตรง Pao168 https://bit.ly/pao13
ถอนกี่ครั้งก็ได้ ถอนได้ไม่อั้น เว็บตรง Pao168 <SHORT-LINK>
['ถอน', 'กี่', 'ครั้ง', 'ก็ได้', 'ถอน', 'ได้', 'ไม่', 'อั้น', 'เว็บ', 'ตรง', 'pao', '<SHORT-LINK>']

1/1 [==============================] - 0s 39ms/step
scam



In [10]:
def load_model(filepath):
    model = models.load_model(filepath)
    return tf.keras.Sequential([model, 
                                tf.keras.layers.Softmax()])

deep_models = {
    'Dense_bow': load_model('models/Dense_bow.h5'),
    'Dense_tfidf': load_model('models/Dense_tfidf.h5'),
    'Dense_seqpad': load_model('models/Dense_seqpad.h5'),
    'BiLSTM_seqpad': load_model('models/BiLSTM_seqpad.h5'),
    
}
for txt in examples:
    
    print (txt)    
    sanitized_txt = textutils.sanitize_text(txt)
    
    for model_name in deep_models:
        prob_model    = None
        textrep_array = None
        
        if model_name.endswith('bow'):
            textrep_array = textutils.bow_array_from_text(sanitized_txt,
                                                          config['word_index'],
                                                          ignore_err=True)

            
        elif model_name.endswith('tfidf'):
            textrep_array = textutils.tfidf_array_from_text(sanitized_txt,
                                                            config['word_index'],
                                                            config['idf_values'],
                                                            ignore_err=True)
        elif model_name.endswith('seqpad'):
            textrep_array = textutils.padded_seq_from_text(sanitized_txt, 
                                                        config['word_index'], 
                                                        config['max_length'], 
                                                        ignore_err=True)
        
        
        
        prob_model = deep_models[model_name]
        predictions = prob_model.predict(np.array([textrep_array]))
        predict = np.argmax(predictions[0])
        
        print (f'{model_name}: {labels[predict]} {predictions}')  

    print ()

Found word not in word_index: word=idn


รหัส OTP สำหรับการยืนยันตัวตนของคุณสำหรับการใช้งาน GHB All คือ 318266 หมายเลขอ้างอิง IDN239S
1/1 [==============================] - 0s 63ms/step


Found word not in word_index: word=idn


Dense_bow: OK/OTP [[0.5744364  0.21169758 0.21386595]]
1/1 [==============================] - 0s 58ms/step


Found word not in word_index: word=idn


...

Dense_bow: scam [[0.23063047 0.20982653 0.559543  ]]
1/1 [==============================] - 0s 22ms/step


Found word not in word_index: word=pao


Dense_tfidf: scam [[0.21713078 0.21621655 0.5666527 ]]
1/1 [==============================] - 0s 21ms/step


Found word not in word_index: word=pao


Dense_seqpad: scam [[0.2151943  0.22693925 0.55786645]]
1/1 [==============================] - 0s 28ms/step
BiLSTM_seqpad: scam [[0.17497379 0.17497012 0.4751427  0.17491339]]

